## Import

In [1]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmcv import Config

import os
os.chdir("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/")
!ls -ail

total 1792
715 drwxr-xr-x  7 bgw2001 bgw2001 262144 11월 28 00:32 .
683 drwxr-xr-x 18 bgw2001 bgw2001 262144 11월 17 17:26 ..
723 drwxr-xr-x  6 bgw2001 bgw2001 262144 11월 30 10:52 code
721 drwxr-xr-x  8 bgw2001 bgw2001 262144 11월 25 11:24 data
726 drwxr-xr-x  2 bgw2001 bgw2001 262144 11월 25 11:40 .ipynb_checkpoints
724 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 28 18:52 practice_code
722 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 19 14:50 ref


In [2]:
# 데이터 셋 등록
@DATASETS.register_module(force = True)
class CapsuleDataset(CocoDataset):
    CLASSES = ("01_ulcer", "02_mass", "04_lymph", "05_bleeding")

In [3]:
#!wget -O /home/bgw2001/mmdetection/checkpoints/cascade_rcnn_r50_sac_1x_coco-24bfda62.pth https://download.openmmlab.com/mmdetection/v2.0/detectors/cascade_rcnn_r50_sac_1x_coco/cascade_rcnn_r50_sac_1x_coco-24bfda62.pth

--2021-11-30 14:38:26--  https://download.openmmlab.com/mmdetection/v2.0/detectors/cascade_rcnn_r50_sac_1x_coco/cascade_rcnn_r50_sac_1x_coco-24bfda62.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.5
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337597436 (322M) [application/octet-stream]
Saving to: ‘/home/bgw2001/mmdetection/checkpoints/cascade_rcnn_r50_sac_1x_coco-24bfda62.pth’

/home/bgw2001/mmdet 100%[===================>] 321.96M  8.03MB/s    in 45s     

2021-11-30 14:39:12 (7.20 MB/s) - ‘/home/bgw2001/mmdetection/checkpoints/cascade_rcnn_r50_sac_1x_coco-24bfda62.pth’ saved [337597436/337597436]



In [4]:
#!ls -ail /home/bgw2001/mmdetection/checkpoints/

total 3514720
52580884 drwxrwxr-x  2 bgw2001 bgw2001      4096 11월 30 14:38 .
50602523 drwxrwxr-x 18 bgw2001 bgw2001      4096 11월 12 13:57 ..
51646618 -rw-rw-r--  1 bgw2001 bgw2001 384456573  9월 11  2020 cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_1x_coco_20200310-d5ad2a5e.pth
52580955 -rw-rw-r--  1 bgw2001 bgw2001 337597436 11월  3 01:11 cascade_rcnn_r50_sac_1x_coco-24bfda62.pth
52580858 -rw-rw-r--  1 bgw2001 bgw2001 496633663 11월  3 01:11 detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth
51646232 -rw-rw-r--  1 bgw2001 bgw2001 167287506  8월 28  2020 faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
50075751 -rw-rw-r--  1 bgw2001 bgw2001 242159529 11월 21 19:51 faster_rcnn_x101_32x4d_fpn_1x_coco_20200203-cff10310.pth
51646567 -rw-rw-r--  1 bgw2001 bgw2001 254089599  8월 28  2020 mask_rcnn_r101_fpn_1x_coco_20200204-1efe0ed5.pth
51646619 -rw-rw-r--  1 bgw2001 bgw2001 273287445  9월 11  2020 mask_rcnn_r101_fpn_syncbn-backbone_r16_gcb_c3-c5_1x_coco_20200207-945e77ca.pth
51646623 -rw-rw-r--

In [3]:
# config file, checkpoints 파일 경로 셋팅
config_file = "/home/bgw2001/mmdetection/configs/detectors/cascade_rcnn_r50_sac_1x_coco.py"
checkpoints_file = "/home/bgw2001/mmdetection/checkpoints/cascade_rcnn_r50_sac_1x_coco-24bfda62.pth"

In [4]:
cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50'),
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True)),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0

In [5]:
from mmdet.apis import set_random_seed

## config file setting
cfg.seed = 0
set_random_seed(0, deterministic=False)

for roi in cfg.model.roi_head.bbox_head:
    roi.num_classes = 4
    
cfg.dataset_type = "CapsuleDataset"
cfg.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"

cfg.data.train.type = 'CapsuleDataset'
cfg.data.train.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.train.ann_file = "train.json"
cfg.data.train.img_prefix = "train"

cfg.data.val.type = 'CapsuleDataset'
cfg.data.val.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.val.ann_file = "valid.json"
cfg.data.val.img_prefix = "valid"

cfg.data.test.type = 'CapsuleDataset'
cfg.data.test.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.test.ann_file = "test.json"
cfg.data.test.img_prefix = "test"

cfg.evaluation.interval = 1
cfg.optimizer.lr = 0.02 / 4

cfg.lr_config.policy = 'step'
cfg.lr_config.warmup = None
cfg.lr_config.step = [16, 22]
cfg.lr_config.gamma = 0.1

cfg.log_config.interval = 100
cfg.runner.max_epochs = 24

cfg.checkpoint_config.interval = 1

cfg.load_from = checkpoints_file
cfg.work_dir = "/home/bgw2001/mmdetection/work_dir/"

cfg.gpu_ids = range(1)
cfg.data.samples_per_gpu = 4

albu_train_transforms = [
    dict(
        type='HorizontalFlip',
        p=0.5)
]

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.train_pipeline = train_pipeline
cfg.data.train.pipeline = train_pipeline

In [6]:
print(cfg.pretty_text)

model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50'),
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True)),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0

In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [8]:
datasets[0]


CapsuleDataset Train dataset with number of images 37573, and instance counts: 
+--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+
| category     | count | category    | count | category     | count | category        | count | category      | count |
+--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+
| 0 [01_ulcer] | 23439 | 1 [02_mass] | 5795  | 2 [04_lymph] | 7156  | 3 [05_bleeding] | 11815 | -1 background | 0     |
+--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+

In [9]:
datasets[0].CLASSES

('01_ulcer', '02_mass', '04_lymph', '05_bleeding')

In [10]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg = cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES
print(model.CLASSES)

('01_ulcer', '02_mass', '04_lymph', '05_bleeding')


In [11]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.07s)
creating index...


2021-11-30 14:45:57,925 - mmdet - INFO - load checkpoint from /home/bgw2001/mmdetection/checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth
2021-11-30 14:45:57,926 - mmdet - INFO - Use load_from_local loader


index created!


2021-11-30 14:45:58,156 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.2.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.2.fc_cls.bias: copying 

2021-11-30 14:45:58,164 - mmdet - INFO - Start running, host: bgw2001@bgw2001-All-Series, work_dir: /home/bgw2001/mmdetection/work_dir
2021-11-30 14:45:58,165 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHoo

...

[>>>>>>>>>>>>>>>>>>>>>>>] 12524/12524, 10.1 task/s, elapsed: 1237s, ETA:     0s

2021-12-03 22:28:21,561 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.40s).
Accumulating evaluation results...


2021-12-03 22:28:26,983 - mmdet - INFO - Epoch(val) [24][12524]	bbox_mAP: 0.4980, bbox_mAP_50: 0.8360, bbox_mAP_75: 0.5220, bbox_mAP_s: 0.2050, bbox_mAP_m: 0.4740, bbox_mAP_l: 0.4960, bbox_mAP_copypaste: 0.498 0.836 0.522 0.205 0.474 0.496


DONE (t=0.83s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.498
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.836
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.522
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.496
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.580
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.580
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.580
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.234
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.560
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.581


In [12]:
cfg.dump("/home/bgw2001/mmdetection/work_dir/detectors_resnet50_conf.py")

 - https://www.kaggle.com/vfomenko/detectors-mmdetection-training

In [1]:
import json
import json
import base64
import numpy as np
import cv2
import pandas as pd
import os

from PIL import Image
from io import BytesIO
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset
from tqdm import tqdm
from glob import glob
from mmcv import Config

In [4]:
os.chdir("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/")
!ls -ail

total 2304
   809 drwxr-xr-x  9 bgw2001 bgw2001 262144 12월  4 11:38 .
   706 drwxr-xr-x 18 bgw2001 bgw2001 262144 11월 17 17:26 ..
  1150 drwxr-xr-x  5 bgw2001 bgw2001 262144 12월  4 11:35 code
  1147 drwxr-xr-x  7 bgw2001 bgw2001 262144 12월  4 11:35 data
  1149 drwxr-xr-x  2 bgw2001 bgw2001 262144 11월 25 11:40 .ipynb_checkpoints
  1151 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 28 18:52 practice_code
  1148 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 19 14:50 ref
  1563 drwxr-xr-x  2 bgw2001 bgw2001 262144 12월  4 11:37 submission
175699 drwxr-xr-x  4 bgw2001 bgw2001 262144 12월  4 11:42 work_dir


### epoch_17

In [5]:
# 데이터 셋 등록
@DATASETS.register_module(force = True)
class CapsuleDataset(CocoDataset):
    CLASSES = ("01_ulcer", "02_mass", "04_lymph", "05_bleeding")

config_file = "./work_dir/DetectoRS/detectors_resnet50_conf.py"
checkpoint_file = "./work_dir/DetectoRS/epoch_17.pth"

In [6]:
def submission(model, paths, score_thr = 0.5):
    file_names = []
    class_ids = []
    confidences = []
    
    point1_x = []
    point1_y = []
    
    point2_x = []
    point2_y = []

    point3_x = []
    point3_y = []

    point4_x = []
    point4_y = []
    
    for path in tqdm(paths):
        
        with open(path, "r") as f:
            data = json.load(f)
        
        img = Image.open(BytesIO(base64.b64decode(data['imageData'])))
        img = np.array(img, np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        results = inference_detector(model, img)
        
        index = path.rfind("/")
        file_name = path[index + 1:]
        
        for ind,result in enumerate(results):
            for i in range(len(result)):
                if result[i,4] > score_thr:
                    file_names.append(file_name)
                    xmin = result[i,0]
                    ymin = result[i,1]
                    xmax = result[i,2]
                    ymax = result[i,3]
            
                    point1_x.append(xmin)
                    point1_y.append(ymin)
            
                    point2_x.append(xmax)
                    point2_y.append(ymin)
            
                    point3_x.append(xmax)
                    point3_y.append(ymax)
            
                    point4_x.append(xmin)
                    point4_y.append(ymax)
            
                    class_ids.append(ind)
                    confidences.append(result[i,4])
    
    submission = pd.DataFrame({"file_name":file_names,
                               "class_id":class_ids,
                               "confidence":confidences,
                               "point1_x":point1_x,
                               "point1_y":point1_y,
                               "point2_x":point2_x,
                               "point2_y":point2_y,
                               "point3_x":point3_x,
                               "point3_y":point3_y,
                               "point4_x":point4_x,
                               "point4_y":point4_y})
    return submission

In [7]:
model = init_detector(config_file, checkpoint_file)

paths = glob("./data/test/*.json")

Use load_from_local loader


In [8]:
print(len(paths))

20874


In [10]:
df = submission(model, paths, score_thr = 0.6)

  0%|                                                                                                                                  | 0/20874 [00:00<?, ?it/s]/home/bgw2001/mmdetection/mmdet/datasets/utils.py:65: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
/home/bgw2001/anaconda3/envs/py3.8/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20874/20874 [38:27<00:00,  9.05

In [11]:
print("데이터 프레임 모양:",df.shape)
df.head()

데이터 프레임 모양: (25867, 11)


,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,3,0.982580,378.866302,345.252350,403.948700,345.252350,403.948700,378.034546,378.866302,378.034546
1,test_200001.json,0,0.996303,391.228577,176.836075,531.433716,176.836075,531.433716,316.476257,391.228577,316.476257
2,test_200001.json,0,0.995174,121.674103,335.397095,263.554688,335.397095,263.554688,473.304352,121.674103,473.304352
3,test_200002.json,1,0.994371,222.788376,345.807098,341.557831,345.807098,341.557831,471.914001,222.788376,471.914001
4,test_200003.json,1,0.998270,230.796600,135.197662,317.902130,135.197662,317.902130,258.696350,230.796600,258.696350


In [12]:
df['class_id'] = df['class_id'] + 1
df.head()

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,4,0.982580,378.866302,345.252350,403.948700,345.252350,403.948700,378.034546,378.866302,378.034546
1,test_200001.json,1,0.996303,391.228577,176.836075,531.433716,176.836075,531.433716,316.476257,391.228577,316.476257
2,test_200001.json,1,0.995174,121.674103,335.397095,263.554688,335.397095,263.554688,473.304352,121.674103,473.304352
3,test_200002.json,2,0.994371,222.788376,345.807098,341.557831,345.807098,341.557831,471.914001,222.788376,471.914001
4,test_200003.json,2,0.998270,230.796600,135.197662,317.902130,135.197662,317.902130,258.696350,230.796600,258.696350


In [13]:
df.to_csv("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/submission/DetectoRS_e17.csv",index=False)

### epoch_18

In [14]:
config_file = "./work_dir/DetectoRS/detectors_resnet50_conf.py"
checkpoint_file = "./work_dir/DetectoRS/epoch_18.pth"
paths = glob("./data/test/*.json")

model = init_detector(config_file, checkpoint_file)
df = submission(model, paths, score_thr = 0.6)

Use load_from_local loader


  0%|                                                                                                                                  | 0/20874 [00:00<?, ?it/s]/home/bgw2001/mmdetection/mmdet/datasets/utils.py:65: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20874/20874 [38:10<00:00,  9.11it/s]


In [15]:
print("데이터 프레임 모양:",df.shape)
df.head()

데이터 프레임 모양: (25914, 11)


,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,3,0.983912,377.866516,345.041962,404.099579,345.041962,404.099579,378.712952,377.866516,378.712952
1,test_200001.json,0,0.998317,392.050598,174.420212,530.656433,174.420212,530.656433,318.213013,392.050598,318.213013
2,test_200001.json,0,0.996189,124.583282,337.258667,262.988373,337.258667,262.988373,472.311859,124.583282,472.311859
3,test_200002.json,1,0.995833,222.380783,345.321625,341.517456,345.321625,341.517456,470.260773,222.380783,470.260773
4,test_200003.json,1,0.998720,230.693527,134.135956,318.112518,134.135956,318.112518,259.632019,230.693527,259.632019


In [16]:
df['class_id'] = df['class_id'] + 1
df.head()

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,4,0.983912,377.866516,345.041962,404.099579,345.041962,404.099579,378.712952,377.866516,378.712952
1,test_200001.json,1,0.998317,392.050598,174.420212,530.656433,174.420212,530.656433,318.213013,392.050598,318.213013
2,test_200001.json,1,0.996189,124.583282,337.258667,262.988373,337.258667,262.988373,472.311859,124.583282,472.311859
3,test_200002.json,2,0.995833,222.380783,345.321625,341.517456,345.321625,341.517456,470.260773,222.380783,470.260773
4,test_200003.json,2,0.998720,230.693527,134.135956,318.112518,134.135956,318.112518,259.632019,230.693527,259.632019


In [17]:
df.to_csv("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/submission/DetectoRS_e18.csv",index=False)

## epoch_19

In [18]:
config_file = "./work_dir/DetectoRS/detectors_resnet50_conf.py"
checkpoint_file = "./work_dir/DetectoRS/epoch_19.pth"
paths = glob("./data/test/*.json")

model = init_detector(config_file, checkpoint_file)
df = submission(model, paths, score_thr = 0.6)

Use load_from_local loader


  0%|                                                                                                                                  | 0/20874 [00:00<?, ?it/s]/home/bgw2001/mmdetection/mmdet/datasets/utils.py:65: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20874/20874 [38:07<00:00,  9.12it/s]


In [19]:
print("데이터 프레임 모양:",df.shape)
df.head()

데이터 프레임 모양: (25240, 11)


,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,3,0.990280,378.100616,345.405914,403.867676,345.405914,403.867676,378.619598,378.100616,378.619598
1,test_200001.json,0,0.998147,393.093170,175.952576,529.426270,175.952576,529.426270,315.259277,393.093170,315.259277
2,test_200001.json,0,0.996974,124.777283,337.656799,261.200928,337.656799,261.200928,470.939880,124.777283,470.939880
3,test_200002.json,1,0.995964,221.924896,345.473541,342.004730,345.473541,342.004730,474.307495,221.924896,474.307495
4,test_200003.json,1,0.999236,230.773285,133.602615,318.079834,133.602615,318.079834,259.554718,230.773285,259.554718


In [20]:
df['class_id'] = df['class_id'] + 1
df.head()

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,4,0.990280,378.100616,345.405914,403.867676,345.405914,403.867676,378.619598,378.100616,378.619598
1,test_200001.json,1,0.998147,393.093170,175.952576,529.426270,175.952576,529.426270,315.259277,393.093170,315.259277
2,test_200001.json,1,0.996974,124.777283,337.656799,261.200928,337.656799,261.200928,470.939880,124.777283,470.939880
3,test_200002.json,2,0.995964,221.924896,345.473541,342.004730,345.473541,342.004730,474.307495,221.924896,474.307495
4,test_200003.json,2,0.999236,230.773285,133.602615,318.079834,133.602615,318.079834,259.554718,230.773285,259.554718


In [21]:
df.to_csv("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/submission/DetectoRS_e19.csv",index=False)